In [4]:
from time import perf_counter
import numpy as np
import cv2
from skimage import exposure
import matplotlib.pyplot as plt
from typing import Any
import os
from glob import glob
from skimage import exposure,measure
from skimage.segmentation import clear_border
import skimage.io
from skimage.filters import threshold_otsu,threshold_triangle,threshold_li,threshold_mean,median
from scipy import ndimage as ndi
import scipy
from skimage.morphology import area_closing,binary_closing,binary_erosion,binary_dilation,binary_opening,\
     remove_small_objects,remove_small_holes,reconstruction,erosion,closing,dilation,opening,disk

from skimage.segmentation import felzenszwalb, slic, quickshift, watershed
from skimage.feature import peak_local_max
from skimage.transform import rotate
import tifffile as tiff
import subprocess
from skimage import measure

In [7]:
dice_score=0

a='F:/celltrackingdata/2D Data/Fluo-N2DH-SIM/01_gt/contain_mitosis/01_GT/seg/'
mask_names1= glob(a + "*.*")

img_num=len(mask_names1)
print('number of images=',img_num)

for imgn in mask_names1:
    #if(imgn[0]==imgn[1]):
    #    continue;
    print(imgn)

    gt1= cv2.imread(imgn,cv2.IMREAD_ANYDEPTH)
    gt1=(gt1>0)*1
    
    imgnn=imgn.replace('01_gt/contain_mitosis\\man_seg','01/t')
    print(imgnn)

    image1=cv2.imread(imgnn,cv2.IMREAD_ANYDEPTH)
    image1=cv2.GaussianBlur(image1,(3,3),0)
    p2, p98 = np.percentile(image1, (0.2, 99.8),method='median_unbiased')
    image1 = skimage.exposure.rescale_intensity(image1, in_range=(p2, p98))
    image1 = (image1 - image1.min()) / (image1.max() - image1.min())
    image1 = skimage.exposure.equalize_adapthist(image1, clip_limit=0.01).astype(np.float32)# Adaptive Equalization
    
    segments_fz = felzenszwalb(image1, scale=100, sigma=0.5, min_size=50)
    thresh = threshold_triangle(segments_fz)
    preds = (segments_fz > thresh)*1

    intersection=(preds * gt1).sum()
    union=(preds + gt1).sum() 
    dice_score += (2 * intersection / union)
   
    res= measure.label(preds, background=0)
    dir1='F:/celltrackingdata/2D Data/Fluo-N2DH-SIM/01_GT/contain_mitosis/mask'+imgn[-7:]
    print(dir1)
    tiff.imsave(dir1, res.astype(np.uint16), compress=1)
    #plt.imshow(image1)
    #plt.show()
    
    #plt.imshow(res1)
    #plt.show()

print('dice=',dice_score/img_num)    

number of images= 27
F:/celltrackingdata/2D Data/Fluo-N2DH-SIM/01_gt/contain_mitosis/01_GT/seg\man_seg009.tif
F:/celltrackingdata/2D Data/Fluo-N2DH-SIM/01_gt/contain_mitosis/01_GT/seg\man_seg009.tif
F:/celltrackingdata/2D Data/Fluo-N2DH-SIM/01_GT/contain_mitosis/mask009.tif
F:/celltrackingdata/2D Data/Fluo-N2DH-SIM/01_gt/contain_mitosis/01_GT/seg\man_seg010.tif
F:/celltrackingdata/2D Data/Fluo-N2DH-SIM/01_gt/contain_mitosis/01_GT/seg\man_seg010.tif
F:/celltrackingdata/2D Data/Fluo-N2DH-SIM/01_GT/contain_mitosis/mask010.tif
F:/celltrackingdata/2D Data/Fluo-N2DH-SIM/01_gt/contain_mitosis/01_GT/seg\man_seg011.tif
F:/celltrackingdata/2D Data/Fluo-N2DH-SIM/01_gt/contain_mitosis/01_GT/seg\man_seg011.tif
F:/celltrackingdata/2D Data/Fluo-N2DH-SIM/01_GT/contain_mitosis/mask011.tif
F:/celltrackingdata/2D Data/Fluo-N2DH-SIM/01_gt/contain_mitosis/01_GT/seg\man_seg013.tif
F:/celltrackingdata/2D Data/Fluo-N2DH-SIM/01_gt/contain_mitosis/01_GT/seg\man_seg013.tif
F:/celltrackingdata/2D Data/Fluo-N2DH-S

In [8]:
dir2='F:/celltrackingdata/2D Data/Fluo-N2DH-SIM/01_GT/contain_mitosis/'

result = subprocess.run(
    ["f:/infer/win/SEGMeasure.exe", dir2,'01', "3"], capture_output=True, text=True )
print("stdout:", result.stdout)

stdout: SEG measure: 0.750561

